In [1]:
import numpy as np
import pandas as pd 
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score,roc_auc_score,make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

In [2]:
dataset=pd.read_excel("Telco_customer_churn.xlsx")

In [3]:
dataset.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,...,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,86,5372,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,...,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,84,5003,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,...,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,89,5340,Competitor had better devices


In [5]:
dataset["Count"].unique()

array([1], dtype=int64)

In [6]:
dataset.drop(["CustomerID","Count","Country","State","Lat Long"],axis=1,inplace=True)

In [7]:
dataset.head(1)

,City,Zip Code,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,Los Angeles,90003,33.964131,-118.272783,Male,No,No,No,2,Yes,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer


In [8]:
dataset["City"].replace(" ","-",regex=True,inplace=True)#regex: It's a advanced featue of for a  search and replace function

In [10]:
dataset.head(2)

,City,Zip Code,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,...,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV,Churn Reason
0,Los-Angeles,90003,33.964131,-118.272783,Male,No,No,No,2,Yes,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,Los-Angeles,90005,34.059281,-118.307420,Female,No,No,Yes,2,Yes,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved


In [18]:
X=dataset["City"].unique()

In [20]:
len(X)

1129

In [21]:
dataset["City"].unique()[0:10]

array(['Los-Angeles', 'Beverly-Hills', 'Huntington-Park', 'Lynwood',
       'Marina-Del-Rey', 'Inglewood', 'Santa-Monica', 'Torrance',
       'Whittier', 'La-Habra'], dtype=object)

In [23]:
dataset.columns=dataset.columns.str.replace(" ","_")
dataset.head(2)

,City,Zip_Code,Latitude,Longitude,Gender,Senior_Citizen,Partner,Dependents,Tenure_Months,Phone_Service,...,Contract,Paperless_Billing,Payment_Method,Monthly_Charges,Total_Charges,Churn_Label,Churn_Value,Churn_Score,CLTV,Churn_Reason
0,Los-Angeles,90003,33.964131,-118.272783,Male,No,No,No,2,Yes,...,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,86,3239,Competitor made better offer
1,Los-Angeles,90005,34.059281,-118.307420,Female,No,No,Yes,2,Yes,...,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,67,2701,Moved


In [28]:
dataset["Total_Charges"].dtype

dtype('O')

In [29]:
dataset.dtypes

City                  object
Zip_Code               int64
Latitude             float64
Longitude            float64
Gender                object
Senior_Citizen        object
Partner               object
Dependents            object
Tenure_Months          int64
Phone_Service         object
Multiple_Lines        object
Internet_Service      object
Online_Security       object
Online_Backup         object
Device_Protection     object
Tech_Support          object
Streaming_TV          object
Streaming_Movies      object
Contract              object
Paperless_Billing     object
Payment_Method        object
Monthly_Charges      float64
Total_Charges         object
Churn_Label           object
Churn_Value            int64
Churn_Score            int64
CLTV                   int64
Churn_Reason          object
dtype: object

In [30]:
dataset["Total_Charges"]=pd.to_numeric(dataset["Total_Charges"])

ValueError: Unable to parse string " " at position 2234

In [31]:
dataset.loc[dataset["Total_Charges"]==" "]

,City,Zip_Code,Latitude,Longitude,Gender,Senior_Citizen,Partner,Dependents,Tenure_Months,Phone_Service,...,Contract,Paperless_Billing,Payment_Method,Monthly_Charges,Total_Charges,Churn_Label,Churn_Value,Churn_Score,CLTV,Churn_Reason
2234,San-Bernardino,92408,34.084909,-117.258107,Female,No,Yes,No,0,No,...,Two year,Yes,Bank transfer (automatic),52.55,,No,0,36,2578,NaN
2438,Independence,93526,36.869584,-118.189241,Male,No,No,No,0,Yes,...,Two year,No,Mailed check,20.25,,No,0,68,5504,NaN
2568,San-Mateo,94401,37.590421,-122.306467,Female,No,Yes,No,0,Yes,...,Two year,No,Mailed check,80.85,,No,0,45,2048,NaN
2667,Cupertino,95014,37.306612,-122.080621,Male,No,Yes,Yes,0,Yes,...,Two year,No,Mailed check,25.75,,No,0,48,4950,NaN
2856,Redcrest,95569,40.363446,-123.835041,Female,No,Yes,No,0,No,...,Two year,No,Credit card (automatic),56.05,,No,0,30,4740,NaN
4331,Los-Angeles,90029,34.089953,-118.294824,Male,No,Yes,Yes,0,Yes,...,Two year,No,Mailed check,19.85,,No,0,53,2019,NaN
4687,Sun-City,92585,33.739412,-117.173334,Male,No,Yes,Yes,0,Yes,...,Two year,No,Mailed check,25.35,,No,0,49,2299,NaN
5104,Ben-Lomond,95005,37.078873,-122.090386,Female,No,Yes,Yes,0,Yes,...,Two year,No,Mailed check,20.00,,No,0,27,3763,NaN
5719,La-Verne,91750,34.144703,-117.770299,Male,No,Yes,Yes,0,Yes,...,One year,Yes,Mailed check,19.70,,No,0,69,4890,NaN
6772,Bell,90201,33.970343,-118.171368,Female,No,Yes,Yes,0,Yes,...,Two year,No,Mailed check,73.35,,No,0,44,2342,NaN


In [38]:
len(dataset.loc[dataset["Total_Charges"]==" "])

11

In [39]:
dataset.isnull().sum()

City                    0
Zip_Code                0
Latitude                0
Longitude               0
Gender                  0
Senior_Citizen          0
Partner                 0
Dependents              0
Tenure_Months           0
Phone_Service           0
Multiple_Lines          0
Internet_Service        0
Online_Security         0
Online_Backup           0
Device_Protection       0
Tech_Support            0
Streaming_TV            0
Streaming_Movies        0
Contract                0
Paperless_Billing       0
Payment_Method          0
Monthly_Charges         0
Total_Charges           0
Churn_Label             0
Churn_Value             0
Churn_Score             0
CLTV                    0
Churn_Reason         5174
dtype: int64

In [41]:
dataset.loc[(dataset["Total_Charges"]==" "),"Total_Charges"]=0

In [42]:
dataset.loc[dataset["Total_Charges"]==" "]

,City,Zip_Code,Latitude,Longitude,Gender,Senior_Citizen,Partner,Dependents,Tenure_Months,Phone_Service,...,Contract,Paperless_Billing,Payment_Method,Monthly_Charges,Total_Charges,Churn_Label,Churn_Value,Churn_Score,CLTV,Churn_Reason


In [43]:
dataset["Total_Charges"]=pd.to_numeric(dataset["Total_Charges"])

In [44]:
dataset.replace(" ","_",regex=True,inplace=True)

In [45]:
dataset.head(5)

,City,Zip_Code,Latitude,Longitude,Gender,Senior_Citizen,Partner,Dependents,Tenure_Months,Phone_Service,...,Contract,Paperless_Billing,Payment_Method,Monthly_Charges,Total_Charges,Churn_Label,Churn_Value,Churn_Score,CLTV,Churn_Reason
0,Los-Angeles,90003,33.964131,-118.272783,Male,No,No,No,2,Yes,...,Month-to-month,Yes,Mailed_check,53.85,108.15,Yes,1,86,3239,Competitor_made_better_offer
1,Los-Angeles,90005,34.059281,-118.307420,Female,No,No,Yes,2,Yes,...,Month-to-month,Yes,Electronic_check,70.70,151.65,Yes,1,67,2701,Moved
2,Los-Angeles,90006,34.048013,-118.293953,Female,No,No,Yes,8,Yes,...,Month-to-month,Yes,Electronic_check,99.65,820.50,Yes,1,86,5372,Moved
3,Los-Angeles,90010,34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,...,Month-to-month,Yes,Electronic_check,104.80,3046.05,Yes,1,84,5003,Moved
4,Los-Angeles,90015,34.039224,-118.266293,Male,No,No,Yes,49,Yes,...,Month-to-month,Yes,Bank_transfer_(automatic),103.70,5036.30,Yes,1,89,5340,Competitor_had_better_devices


In [46]:
X=dataset.drop("Churn_Value",axis=1)
Y=dataset["Churn_Value"]

In [50]:
Y.head()

0    1
1    1
2    1
3    1
4    1
Name: Churn_Value, dtype: int64

In [51]:
pd.get_dummies(X,columns=["Payment_Method"]).head()

,City,Zip_Code,Latitude,Longitude,Gender,Senior_Citizen,Partner,Dependents,Tenure_Months,Phone_Service,...,Monthly_Charges,Total_Charges,Churn_Label,Churn_Score,CLTV,Churn_Reason,Payment_Method_Bank_transfer_(automatic),Payment_Method_Credit_card_(automatic),Payment_Method_Electronic_check,Payment_Method_Mailed_check
0,Los-Angeles,90003,33.964131,-118.272783,Male,No,No,No,2,Yes,...,53.85,108.15,Yes,86,3239,Competitor_made_better_offer,0,0,0,1
1,Los-Angeles,90005,34.059281,-118.307420,Female,No,No,Yes,2,Yes,...,70.70,151.65,Yes,67,2701,Moved,0,0,1,0
2,Los-Angeles,90006,34.048013,-118.293953,Female,No,No,Yes,8,Yes,...,99.65,820.50,Yes,86,5372,Moved,0,0,1,0
3,Los-Angeles,90010,34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,...,104.80,3046.05,Yes,84,5003,Moved,0,0,1,0
4,Los-Angeles,90015,34.039224,-118.266293,Male,No,No,Yes,49,Yes,...,103.70,5036.30,Yes,89,5340,Competitor_had_better_devices,1,0,0,0


In [61]:
X_encoded=pd.get_dummies(X,columns=["City","Gender","Senior_Citizen","Partner","Dependents","Phone_Service","Multiple_Lines",
                                   "Internet_Service","Online_Security","Online_Backup","Device_Protection","Tech_Support","Streaming_TV","Contract",
                                   "Paperless_Billing","Payment_Method","Churn_Label","Churn_Reason"])

In [60]:
dataset.dtypes

City                  object
Zip_Code               int64
Latitude             float64
Longitude            float64
Gender                object
Senior_Citizen        object
Partner               object
Dependents            object
Tenure_Months          int64
Phone_Service         object
Multiple_Lines        object
Internet_Service      object
Online_Security       object
Online_Backup         object
Device_Protection     object
Tech_Support          object
Streaming_TV          object
Streaming_Movies      object
Contract              object
Paperless_Billing     object
Payment_Method        object
Monthly_Charges      float64
Total_Charges        float64
Churn_Label           object
Churn_Value            int64
Churn_Score            int64
CLTV                   int64
Churn_Reason          object
dtype: object

In [62]:
X_encoded.head()

,Zip_Code,Latitude,Longitude,Tenure_Months,Streaming_Movies,Monthly_Charges,Total_Charges,Churn_Score,CLTV,City_Acampo,...,Churn_Reason_Lack_of_self-service_on_Website,Churn_Reason_Limited_range_of_services,Churn_Reason_Long_distance_charges,Churn_Reason_Moved,Churn_Reason_Network_reliability,Churn_Reason_Poor_expertise_of_online_support,Churn_Reason_Poor_expertise_of_phone_support,Churn_Reason_Price_too_high,Churn_Reason_Product_dissatisfaction,Churn_Reason_Service_dissatisfaction
0,90003,33.964131,-118.272783,2,No,53.85,108.15,86,3239,0,...,0,0,0,0,0,0,0,0,0,0
1,90005,34.059281,-118.307420,2,No,70.70,151.65,67,2701,0,...,0,0,0,1,0,0,0,0,0,0
2,90006,34.048013,-118.293953,8,Yes,99.65,820.50,86,5372,0,...,0,0,0,1,0,0,0,0,0,0
3,90010,34.062125,-118.315709,28,Yes,104.80,3046.05,84,5003,0,...,0,0,0,1,0,0,0,0,0,0
4,90015,34.039224,-118.266293,49,Yes,103.70,5036.30,89,5340,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
sum(Y)/len(Y)

0.2653698707936959

In [65]:
Y.avg()

AttributeError: 'Series' object has no attribute 'avg'

In [70]:
X_train,X_test,y_train,y_test=train_test_split(X_encoded,Y,random_state=42,stratify=Y)